In [1]:
import os, glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import imageio
import geopandas as gpd
from IPython.display import Image, display
import tensorflow as tf
from tensorflow.keras import layers, callbacks, backend, Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
labels = gpd.read_file(r"F:\CNOOC_testing\Methane_dataset\labels.geojson")

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [21]:
labels_unique = labels[(labels.Type == 'Negative') | (labels.Type == 'CAFOs') | (labels.Type == 'WWTreatment')
           | (labels.Type == 'Landfills') | (labels.Type == 'RefineriesAndTerminals')
           | (labels.Type == 'ProcessingPlants') | (labels.Type == 'Mines')]

In [19]:
labels_unique.shape

(14126, 7)

In [22]:
X = []
for im_path in labels_unique.Image_Folder.values:
     X.append(imageio.imread(f'F:\\CNOOC_testing\\Methane_dataset\\train_images.tar\\{im_path}/naip.png',pilmode="RGB"))


# RGB Image of 3 samples
#fig, axs = plt.subplots(1, 3, sharey=True, figsize=(12,5))
#axs[0].imshow(X[0][:,:,:3])
#axs[1].imshow(X[10][:,:,:3])
#axs[2].imshow(X[1000][:,:,:3])



#m = folium.Map([labels_unique.Latitude.min(), labels_unique.Longitude.min()], zoom_start=5, tiles='cartodbpositron')
#folium.GeoJson(labels_unique.geometry.iloc[0:200]).add_to(m)



mlb = MultiLabelBinarizer()
y = mlb.fit_transform(labels_unique.Type.values.reshape(labels_unique.Type.values.shape[0], 1))

In [23]:
x_train, x_rest, y_train, y_rest = train_test_split(X, y, train_size=0.7)
x_val, x_test, y_val, y_test = train_test_split(x_rest, y_rest, train_size=0.5)

In [ ]:
X=np.array(X)
y=np.array(y)
np.save(r"D:\New folder\x_ndarray",X)
np.save(r"D:\New folder\y_ndarray",y)

In [2]:
X=np.load(r"F:\CNOOC_testing\Methane_dataset\x_ndarray.npy")
y=np.load(r"F:\CNOOC_testing\Methane_dataset\y_ndarray.npy")

In [7]:
#X_train, X_rest, y_train, y_rest = train_test_split(X, y, train_size=0.7, stratify=y)
#X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, train_size=0.5, stratify=y_rest)
X_train, X_rest, y_train, y_rest = train_test_split(X[:50], y[:50], train_size=0.7)
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, train_size=0.5)

In [24]:
def build_model():
    i = layers.Input(shape=(720,720, 3), dtype = tf.float32)
    i1 = layers.Resizing(
        224,
        224,
        interpolation='bilinear',
        crop_to_aspect_ratio=False
    )(i)
    out = tf.keras.applications.DenseNet121(include_top=True,
                      weights='imagenet'
                      )(i1)
    out = layers.Dense(32, activation='relu')(out)
    out = layers.Dense(7, activation='sigmoid')(out)
    model = Model(inputs=i, outputs=out)
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [ ]:
def build_model():
    i = layers.Input(shape=(720,720, 3))
    out = tf.keras.applications.DenseNet121(include_top=True,
                      weights='imagenet',
                      input_tensor=i)(i)
    out = layers.Dense(32, activation='relu')(out)
    out = layers.Dense(7, activation='sigmoid')(out)
    model = Model(inputs=i, outputs=out)
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [29]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)


In [37]:
backend.clear_session()
model = build_model()
model.layers[2].trainable=False
model.summary()


cb = [callbacks.EarlyStopping(patience=5), callbacks.ReduceLROnPlateau(patience=3),callbacks.ModelCheckpoint('methane.hdf5', monitor='loss',verbose=1, save_best_only=True)]
epochs = 10
x_train=np.array(x_train)
# fits the model on batches with real-time data augmentation:
history =model.fit(datagen.flow(x_train, y_train, batch_size=16,
         subset='training'),
         validation_data=datagen.flow(x_train, y_train,
         batch_size=8, subset='validation'),
         steps_per_epoch=len(x_train) / 16, epochs=epochs)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 720, 720, 3)]     0         
_________________________________________________________________
resizing (Resizing)          (None, 224, 224, 3)       0         
_________________________________________________________________
densenet121 (Functional)     (None, 1000)              8062504   
_________________________________________________________________
dense (Dense)                (None, 32)                32032     
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 231       
Total params: 8,094,767
Trainable params: 32,263
Non-trainable params: 8,062,504
_________________________________________________________________


NameError: name 'x_train' is not defined

In [4]:
labels

,id,Latitude,Longitude,Type,Source,Image_Folder,geometry
0,1,43.004434,-78.208900,Negative,Aeroplanes,train_images/43.00443446102501_-78.20890003579709,"POLYGON ((-78.20567 43.00681, -78.20567 43.002..."
1,6,42.033642,-97.719996,Negative,"White Spec (Seadoo, pivot irrigation, etc.)",train_images/42.03364226519898_-97.7199955063882,"POLYGON ((-97.71676 42.03605, -97.71676 42.031..."
2,9,34.495000,-97.118000,WWTreatment,HydroWASTE,train_images/34.495_-97.118,"POLYGON ((-97.11477 34.49768, -97.11477 34.492..."
3,12,38.381380,-75.401000,CAFOs,DataForCauseChallenge,train_images/38.38138_-75.401,"POLYGON ((-75.39777 38.38393, -75.39777 38.378..."
4,17,45.647973,-84.469159,RefineriesAndTerminals,EIA-HIFLD,train_images/45.647973_-84.469159,"POLYGON ((-84.46593 45.65024, -84.46593 45.645..."
...,...,...,...,...,...,...,...
14182,84941,35.852756,-101.538265,Negative,Windmills,train_images/35.85275555452138_-101.5382646083...,"POLYGON ((-101.53503 35.85539, -101.53503 35.8..."
14183,85008,47.186826,-122.436115,Negative,Roundabout,train_images/47.18682636726484_-122.4361145478...,"POLYGON ((-122.43288 47.18903, -122.43288 47.1..."
14184,85035,42.703517,-83.303270,Negative,Parking Lots,train_images/42.703517482995565_-83.3032698788...,"POLYGON ((-83.30004 42.70590, -83.30004 42.701..."
14185,85048,34.159700,-86.468760,CAFOs,DataForCauseChallenge,train_images/34.1597_-86.46876,"POLYGON ((-86.46553 34.16239, -86.46553 34.157..."


In [15]:
from keras.preprocessing import image
dir_path=r"F:\\CNOOC_testing\\Methane_dataset\\train_images.tar\\"
datagen=image.ImageDataGenerator(rescale=1./255, validation_split = 0.1)

In [22]:
labels_unique.Image_Folder=labels_unique.Image_Folder+"\\naip.png"

e:\program files\python36\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [23]:
labels_unique

,id,Latitude,Longitude,Type,Source,Image_Folder,geometry
0,1,43.004434,-78.208900,Negative,Aeroplanes,train_images/43.00443446102501_-78.20890003579...,"POLYGON ((-78.20567 43.00681, -78.20567 43.002..."
1,6,42.033642,-97.719996,Negative,"White Spec (Seadoo, pivot irrigation, etc.)",train_images/42.03364226519898_-97.71999550638...,"POLYGON ((-97.71676 42.03605, -97.71676 42.031..."
2,9,34.495000,-97.118000,WWTreatment,HydroWASTE,train_images/34.495_-97.118\naip.png,"POLYGON ((-97.11477 34.49768, -97.11477 34.492..."
3,12,38.381380,-75.401000,CAFOs,DataForCauseChallenge,train_images/38.38138_-75.401\naip.png,"POLYGON ((-75.39777 38.38393, -75.39777 38.378..."
4,17,45.647973,-84.469159,RefineriesAndTerminals,EIA-HIFLD,train_images/45.647973_-84.469159\naip.png,"POLYGON ((-84.46593 45.65024, -84.46593 45.645..."
...,...,...,...,...,...,...,...
14182,84941,35.852756,-101.538265,Negative,Windmills,train_images/35.85275555452138_-101.5382646083...,"POLYGON ((-101.53503 35.85539, -101.53503 35.8..."
14183,85008,47.186826,-122.436115,Negative,Roundabout,train_images/47.18682636726484_-122.4361145478...,"POLYGON ((-122.43288 47.18903, -122.43288 47.1..."
14184,85035,42.703517,-83.303270,Negative,Parking Lots,train_images/42.703517482995565_-83.3032698788...,"POLYGON ((-83.30004 42.70590, -83.30004 42.701..."
14185,85048,34.159700,-86.468760,CAFOs,DataForCauseChallenge,train_images/34.1597_-86.46876\naip.png,"POLYGON ((-86.46553 34.16239, -86.46553 34.157..."


In [24]:
train_generator=datagen.flow_from_dataframe(
    dataframe=labels_unique, directory=dir_path, 
    x_col="Image_Folder", y_col="Type", seed = 42,
    class_mode="categorical", target_size=(720,720), batch_size=32, subset = "training")

Found 12714 validated image filenames belonging to 7 classes.


In [25]:
validation_generator = datagen.flow_from_dataframe(dataframe=labels_unique, directory=dir_path, 
    x_col="Image_Folder", y_col="Type", seed = 42,
    class_mode="categorical", target_size=(720,720), batch_size=32, subset = "validation")

Found 1412 validated image filenames belonging to 7 classes.


In [26]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

In [34]:
def build_model():
    i = layers.Input(shape=(720,720, 3), dtype = tf.float32)
    i1 = layers.Resizing(
        224,
        224,
        interpolation='bilinear',
        crop_to_aspect_ratio=False
    )(i)
    out = tf.keras.applications.DenseNet121(include_top=True,
                      weights='imagenet'
                      )(i1)
    out = layers.Dense(32, activation='relu')(out)
    out = layers.Dense(7, activation='sigmoid')(out)
    model = Model(inputs=i, outputs=out)
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
    return model

In [35]:
backend.clear_session()
model = build_model()
model.layers[2].trainable=False
model.summary()


cb = [callbacks.EarlyStopping(patience=5), callbacks.ReduceLROnPlateau(patience=3),callbacks.ModelCheckpoint('methane_V1.hdf5', monitor='loss',verbose=1, save_best_only=True)]
epochs = 50
# fits the model on batches with real-time data augmentation:
history =model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10, callbacks = cb)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 720, 720, 3)]     0         
_________________________________________________________________
resizing (Resizing)          (None, 224, 224, 3)       0         
_________________________________________________________________
densenet121 (Functional)     (None, 1000)              8062504   
_________________________________________________________________
dense (Dense)                (None, 32)                32032     
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 231       
Total params: 8,094,767
Trainable params: 32,263
Non-trainable params: 8,062,504
_________________________________________________________________
Epoch 1/10
397/397 [==============================] - 444s 1s/step - loss: 0.4044 - accuracy: 0.4049 - precision

In [36]:
history.history

{'loss': [0.404377281665802,
  0.3270623981952667,
  0.3270488679409027,
  0.326990008354187,
  0.32697832584381104,
  0.3195992708206177,
  0.28829482197761536,
  0.2757115960121155,
  0.26287898421287537,
  0.24528905749320984],
 'accuracy': [0.4049046039581299,
  0.41846710443496704,
  0.4183882772922516,
  0.41870367527008057,
  0.41830942034721375,
  0.446774959564209,
  0.551490306854248,
  0.5706512928009033,
  0.5980917811393738,
  0.6278977990150452],
 'precision': [0.4000000059604645,
  0.0,
  0.0,
  0.0,
  0.0,
  0.5740131735801697,
  0.659639835357666,
  0.692216694355011,
  0.7408051490783691,
  0.7625771164894104],
 'recall': [0.013404825702309608,
  0.0,
  0.0,
  0.0,
  0.0,
  0.02751931920647621,
  0.3725753128528595,
  0.4151553511619568,
  0.4367607533931732,
  0.506781280040741],
 'val_loss': [0.34214794635772705,
  0.3416500389575958,
  0.33980709314346313,
  0.3416045606136322,
  0.3420332372188568,
  0.32556188106536865,
  0.3622893989086151,
  0.3772841691970825,